In [1]:
%load_ext autoreload 
%autoreload 2

import pickle as pkl
import numpy as np
from autodiff.activation import ReLU, Linear
from autodiff.network import Network, NetworkParams

file_path = 'test-parameters.pkl'

with open(file_path, 'rb') as file:
    data = pkl.load(file)

In [2]:
network_def: NetworkParams = {
    "input_shape": 2,
    "output_shape": 1,
    "layers": [
        {
            "input_shape": 2,
            "n_neurons": len(data['w1']),
            "weight_init": data['w1'],
            "bias_init": data['b1'] ,
            "activation": ReLU(),
        },
        {
            "input_shape": len(data['w1']),
            "n_neurons": len(data['w2']),
            "weight_init": data['w2'],
            "bias_init": data['b2'] ,
            "activation": ReLU(),
        },
        {
            "input_shape": len(data['w2']),
            "n_neurons": len(data['w3']),
            "weight_init": data['w3'],
            "bias_init": data['b3'] ,
            "activation": Linear(), 
        }
    ]
} 


In [4]:
!pip install torch

  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 MB 10.5 MB/s eta 0:00:0000:0100:01
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
import numpy as np
from model import SimpleNet, set_model_weights
import torch
import torch.nn as nn

input_data = [2,3]
target_data = [1]

def get_loss(y, y_hat): 
    return 0.5*((y_hat - y)**2)

# custom network
network = Network(network_def)
y_pred = network.forward(input_data)
y = target_data
loss = get_loss(y, y_pred)
network.backward(y_pred - y) 
print("Custom Backprop Grads")
print("#"*20)
print("First Layer w_grads:   ", network.layers[0].w_grads)
print("First Layer b_grads:   ", network.layers[0].b_grads)
print("#"*20)
criterion = nn.MSELoss()

input_data = torch.tensor([2,3], dtype=torch.float64)
target_data = torch.tensor([1], dtype=torch.float64)

# pytorch network
model = SimpleNet()
set_model_weights(data,model)
output = model(input_data)
loss = criterion(output, target_data)/2

loss.backward()

Custom Backprop Grads
####################
First Layer w_grads:    [[-0.18804252 -0.28206378]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.53346761  0.80020142]
 [ 0.14224417  0.21336625]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [-0.02312626 -0.0346894 ]
 [ 0.          0.        ]]
First Layer b_grads:    [-0.09402126  0.          0.          0.26673381  0.07112208  0.
  0.          0.         -0.01156313  0.        ]
####################


In [7]:
# Check what get_gradients() returns
gradients = model.get_gradients()
print(type(gradients))  # Check the type
print(gradients)         # Print to see the structure


fc1_weight_grad: tensor([[-0.1880, -0.2821],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.5335,  0.8002],
        [ 0.1422,  0.2134],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [-0.0231, -0.0347],
        [ 0.0000,  0.0000]], dtype=torch.float64)
fc1_bias_grad: tensor([-0.0940,  0.0000,  0.0000,  0.2667,  0.0711,  0.0000,  0.0000,  0.0000,
        -0.0116,  0.0000], dtype=torch.float64)
<class 'tuple'>
(tensor([[-0.1880, -0.2821],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.5335,  0.8002],
        [ 0.1422,  0.2134],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [-0.0231, -0.0347],
        [ 0.0000,  0.0000]], dtype=torch.float64), tensor([-0.0940,  0.0000,  0.0000,  0.2667,  0.0711,  0.0000,  0.0000,  0.0000,
        -0.0116,  0.0000], dtype=torch.float64))


In [16]:
# Extract the weight gradients (first element of the tuple)
pytorch_w_gradients = model.get_gradients()[0].detach().cpu().numpy()

# Custom backpropagation gradients (already given)
custom_w_gradients = np.array([
    [-0.18804252, -0.28206378],
    [0, 0],
    [0, 0],
    [0.53346761, 0.80020142],
    [0.14224417, 0.21336625],
    [0, 0],
    [0, 0],
    [0, 0],
    [-0.02312626, -0.0346894],
    [0, 0]
])

# Comparison function
def compare_gradients(pytorch_gradients: np.ndarray, custom_backprop_gradients: np.ndarray, tolerance: float) -> bool:
    if pytorch_gradients.shape != custom_backprop_gradients.shape:
        raise ValueError('The shapes do not match')

    return np.allclose(pytorch_gradients, custom_backprop_gradients, atol=tolerance)

# Run the comparison
result = compare_gradients(
    pytorch_gradients=pytorch_w_gradients,
    custom_backprop_gradients=custom_w_gradients,
    tolerance=0.001
)

print(f"Gradients match: {result}")


fc1_weight_grad: tensor([[-0.1880, -0.2821],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.5335,  0.8002],
        [ 0.1422,  0.2134],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [-0.0231, -0.0347],
        [ 0.0000,  0.0000]], dtype=torch.float64)
fc1_bias_grad: tensor([-0.0940,  0.0000,  0.0000,  0.2667,  0.0711,  0.0000,  0.0000,  0.0000,
        -0.0116,  0.0000], dtype=torch.float64)
[[-0.18804252 -0.28206378]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.53346761  0.80020142]
 [ 0.14224417  0.21336625]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [-0.02312626 -0.0346894 ]
 [ 0.          0.        ]]
Gradients match: True
